In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Import Libraries
# Importing packages and libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#Load new dataset
data = pd.read_csv('/content/gdrive/My Drive/PhD/Program_Doc/Assignment/Project_Dataset/Cleaned_Neg1.csv')
data.head()

,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin..."
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital..."
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '..."
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '..."
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '..."


In [ ]:
len(data.index)

18393

In [ ]:
#New Finding Topics
#Removal of stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
import gensim
from gensim.utils import simple_preprocess

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Word to look for
new = ("covid19 vaccine", "covid19", "vaccine","vaccines", "coronavirus", "vaccination", "covid", "corona","covid19vaccines","covid19 vaccines"
      , "virus", "covid vaccine", "covidvaccine","covid vaccines", "covidvaccines", "covid19vaccine","covidpassport","covid passport","covidpassports","covid passports",
       "covid pass","covidpass","vaccinepassport","vaccine passport",
       "vaccinepassports","vaccine passports","passports","passport","vaccinepass","vaccine pass")
stop.extend(new)
stop[-10:]

['covid pass',
 'covidpass',
 'vaccinepassport',
 'vaccine passport',
 'vaccinepassports',
 'vaccine passports',
 'passports',
 'passport',
 'vaccinepass',
 'vaccine pass']

In [ ]:
#Remove Stopwords
data['Tokens']  = data['Text_lemmatized'] .apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
data.head()

,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized,Tokens
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin...","['here', 'war', 'criminal', 'pushing', 'vaccin..."
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital..."
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...","['agenda', 'obvious', 'eye', 'see', 'check', '..."
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '..."
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...","['agenda', 'obvious', 'eye', 'see', 'check', '..."


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
data_words = list(sent_to_words(data['Tokens']))

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=id2word, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.005*"covid" + 0.004*"vaccine" + 0.003*"people" + 0.003*"fuck" + 0.003*"passport" + 0.002*"right" + 0.002*"nazi" + 0.002*"get" + 0.002*"vaccination" + 0.002*"greatreset"
Topic: 1 Word: 0.006*"vaccine" + 0.006*"covid" + 0.004*"passport" + 0.003*"travel" + 0.003*"dubai" + 0.003*"airport" + 0.003*"bos" + 0.003*"people" + 0.003*"stop" + 0.003*"say"
Topic: 2 Word: 0.006*"vaccine" + 0.004*"covid" + 0.004*"people" + 0.003*"passport" + 0.003*"texas" + 0.003*"ban" + 0.003*"say" + 0.003*"state" + 0.003*"vaccinated" + 0.002*"digital"
Topic: 3 Word: 0.005*"vaccine" + 0.004*"oregon" + 0.004*"covid" + 0.004*"protest" + 0.003*"people" + 0.003*"passport" + 0.003*"need" + 0.002*"racist" + 0.002*"vaccination" + 0.002*"dont"
Topic: 4 Word: 0.005*"covid" + 0.004*"vaccine" + 0.004*"people" + 0.003*"want" + 0.003*"vaccinated" + 0.003*"get" + 0.002*"said" + 0.002*"government" + 0.002*"dont" + 0.002*"vaccination"
Topic: 5 Word: 0.004*"vaccine" + 0.004*"covid" + 0.004*"right" + 0.003*"passport"

In [ ]:
data['Text_lemmatized1'] = data['Text_lemmatized'].str.replace('[^\w\s]','')
#data['Text_cleaned'] = data['Text_lemmatized'].str.replace('[^\w\s]','')
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized,Tokens,Text_lemmatized1
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin...","['here', 'war', 'criminal', 'pushing', 'vaccin...",here war criminal pushing vaccinepassports eve...
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...",sinister huxlean background digitalpassportsva...
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right v...
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...",love idea losing basic human right twat go hac...
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right v...


In [ ]:
#Save new data to file
data.to_csv('/content/gdrive/My Drive/PhD/Program_Doc/Assignment/Project_Dataset/Cleaned_Tweets.csv', encoding='utf-8', index=False)

In [ ]:
#Removal of stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
import gensim
from gensim.utils import simple_preprocess

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Word to look for
new = ("covid19 vaccine", "covid19", "vaccine","vaccines", "coronavirus", "vaccination", "covid", "corona","covid19vaccines","covid19 vaccines"
      , "virus", "covid vaccine", "covidvaccine","covid vaccines", "covidvaccines", "covid19vaccine","covidpassport","covid passport","covidpassports","covid passports",
       "covid pass","covidpass","vaccinepassport","vaccine passport",
       "vaccinepassports","vaccine passports","passports","passport","vaccinepass","vaccine pass")
stop.extend(new)
stop[-10:]

['covid pass',
 'covidpass',
 'vaccinepassport',
 'vaccine passport',
 'vaccinepassports',
 'vaccine passports',
 'passports',
 'passport',
 'vaccinepass',
 'vaccine pass']

In [ ]:
#Remove Stopwords
data['Tokens']  = data['Text_lemmatized1'] .apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
data.head()

,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized,Tokens,Text_lemmatized1
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin...",war criminal pushing everyday life going hard ...,here war criminal pushing vaccinepassports eve...
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...",sinister huxlean background digitalpassportsva...,sinister huxlean background digitalpassportsva...
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...",love idea losing basic human right twat go hac...,love idea losing basic human right twat go hac...
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...


In [ ]:
cv = CountVectorizer(max_df=0.75, min_df=2)
dtm = cv.fit_transform(data['Tokens'])


In [ ]:
print(dtm)


  (0, 12108)	1
  (0, 2714)	1
  (0, 8789)	1
  (0, 3962)	1
  (0, 6423)	1
  (0, 4814)	1
  (0, 5063)	1
  (0, 8136)	1
  (1, 9976)	1
  (1, 5374)	1
  (1, 1094)	1
  (1, 3217)	1
  (2, 540)	1
  (2, 7656)	1
  (2, 4089)	1
  (2, 9705)	1
  (2, 1933)	1
  (2, 2883)	1
  (2, 5084)	1
  (2, 9351)	1
  (3, 9351)	1
  (3, 6583)	1
  (3, 5417)	1
  (3, 6572)	1
  (3, 1162)	1
  :	:
  (18389, 12460)	1
  (18389, 2186)	1
  (18389, 10888)	2
  (18389, 241)	1
  (18389, 9896)	1
  (18389, 10879)	1
  (18389, 12451)	1
  (18389, 9609)	1
  (18389, 2075)	1
  (18389, 4806)	1
  (18390, 1724)	1
  (18390, 1726)	1
  (18390, 1725)	1
  (18390, 1728)	1
  (18391, 1724)	1
  (18391, 1726)	1
  (18391, 1725)	1
  (18391, 1728)	1
  (18392, 8136)	2
  (18392, 6725)	1
  (18392, 10891)	1
  (18392, 12338)	1
  (18392, 742)	1
  (18392, 283)	1
  (18392, 9791)	1


In [ ]:
# We can also get all those words using the get_feature_names() function
feature_names = cv.get_feature_names()
len(feature_names) # show the total number of distinct words

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


12571

In [ ]:
temp_file = pd.DataFrame(feature_names)
len(temp_file[0].unique())

12571

In [ ]:
feature_names

['000',
 '0009',
 '001',
 '003',
 '004',
 '01',
 '015',
 '01_the',
 '02',
 '02042021',
 '05',
 '08105',
 '085105the',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100covidemergency',
 '100k',
 '101',
 '1030',
 '10am',
 '10k',
 '10pm',
 '10th',
 '10will',
 '11',
 '1100',
 '1113junevaccinepassports',
 '114',
 '119',
 '11am',
 '11k',
 '11th',
 '11will',
 '12',
 '120',
 '1250',
 '127k',
 '12m',
 '12th',
 '12will',
 '13',
 '130',
 '130000',
 '134',
 '13k',
 '13lockdownprotest',
 '13will',
 '14',
 '145',
 '14731',
 '14lockdownprotest',
 '14th',
 '14will',
 '15',
 '15000',
 '150000',
 '1524',
 '1525',
 '153342',
 '15450',
 '15600',
 '15th',
 '15will',
 '16',
 '160000',
 '1615',
 '169',
 '16will',
 '17',
 '17000',
 '175',
 '17lockdownprotest',
 '17th',
 '17will',
 '18',
 '180',
 '181',
 '1829',
 '1839s',
 '1897',
 '18will',
 '19',
 '190',
 '1911',
 '1911the',
 '1922',
 '1930s',
 '1933',
 '1938',
 '1939',
 '1940s',
 '1945',
 '1947',
 '1948',
 '195',
 '1957',
 '1958',
 '1972',
 '1984',
 '1990'

In [ ]:
#Apply LDA to find themes/topic
NUM_TOPICS = 7
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=5, random_state=42)

In [ ]:
LDA_model.fit(dtm)

LatentDirichletAllocation(max_iter=5, n_components=7, random_state=42)

In [ ]:
#stored words
len(feature_names)
import random 
for index in range(15):
    random_word_ID = random.randint(0, 650)
    print(cv.get_feature_names()[random_word_ID])

airborne
896
2019
1947
600
agree
agenda1
admet
abusive
ah
200
995
afraid
83
ac


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Top topic per word
len(LDA_model.components_[0])

12571

In [ ]:
# Pick a single topic 
a_topic = LDA_model.components_[0]

# Get the indices that would sort this array
a_topic.argsort()

array([12446,  7420,  8716, ...,  7308,  3433, 10377])

In [ ]:
top_10_words_indices = a_topic.argsort()[-10:]

for i in top_10_words_indices:
    print(cv.get_feature_names()[i])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


go
think
protest
one
say
people
want
need
dont
stop


In [ ]:
for i, topic in enumerate(LDA_model.components_):
    print("THE TOP {} WORDS FOR TOPIC #{}".format(10, i))
    print([cv.get_feature_names()[index] for index in topic.argsort()[-20:]])
    #print("\n")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


THE TOP 10 WORDS FOR TOPIC #0
['history', 'kid', 'public', 'must', 'right', 'france', 'know', 'vaccinated', 'mandatory', 'canada', 'go', 'think', 'protest', 'one', 'say', 'people', 'want', 'need', 'dont', 'stop']
THE TOP 10 WORDS FOR TOPIC #1
['medicaltyranny', 'negative', 'given', 'vaccinemandates', 'republican', 'democrat', 'angry', 'risk', 'need', 'health', 'protect', 'lockdown', 'yet', 'nothing', 'test', 'get', 'right', 'going', 'vaccinated', 'people']
THE TOP 10 WORDS FOR TOPIC #2
['great', 'take', 'forced', 'racist', 'life', 'uk', 'greatreset', 'work', 'card', 'back', 'business', 'lockdown', 'freedom', 'get', 'right', 'want', 'people', 'nazi', 'dont', 'id']
THE TOP 10 WORDS FOR TOPIC #3
['think', 'know', 'anti', 'see', 'business', 'get', 'still', 'day', 'right', 'pandemic', 'many', 'would', 'government', 'make', 'time', 'say', 'vaccinated', 'one', 'need', 'people']
THE TOP 10 WORDS FOR TOPIC #4
['dont', 'today', 'impfpass', 'even', 'thank', 'idea', 'right', 'totalitarianism', 'pa

In [ ]:
final_topics = LDA_model.transform(dtm)
final_topics.shape

(18393, 7)

In [ ]:
final_topics[0]
final_topics[0].argmax()

2

In [ ]:
#Combine topics to original tweets
data["Topic N°"] = final_topics.argmax(axis=1)
data.head()

,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized,Tokens,Text_lemmatized1,Topic N°
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin...",war criminal pushing everyday life going hard ...,here war criminal pushing vaccinepassports eve...,2
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...",sinister huxlean background digitalpassportsva...,sinister huxlean background digitalpassportsva...,5
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...,0
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...",love idea losing basic human right twat go hac...,love idea losing basic human right twat go hac...,0
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...,0


In [ ]:
#Save new data to file
data.to_csv('/content/gdrive/My Drive/PhD/Program_Doc/Assignment/Project_Dataset/Topics_Neg7.csv', encoding='utf-8', index=False)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.sklearn
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
pyLDAvis.enable_notebook() # To enable the visualization on the notebook

In [ ]:
panel = pyLDAvis.sklearn.prepare(LDA_model, dtm, cv, mds='tsne') # Create the panel for the visualization
panel


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6     -29.594002  60.112713       1        1  15.827143
5     -45.996586  11.891685       2        1  15.448405
0      20.368856  70.019325       3        1  14.792048
2      53.928768  31.704309       4        1  13.565197
1     -12.437127 -26.423267       5        1  13.546463
4       3.965971  21.797720       6        1  13.430892
3      37.524284 -16.517805       7        1  13.389853, topic_info=           Term        Freq       Total Category  logprob  loglift
7282       nazi  325.000000  325.000000  Default  30.0000  30.0000
8689    protest  534.000000  534.000000  Default  29.0000  29.0000
5412         id  487.000000  487.000000  Default  28.0000  28.0000
1720     canada  298.000000  298.000000  Default  27.0000  27.0000
716       angry  165.000000  165.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
12382     world   89.196859  525.954171   Topic7  -5.8257   0.2363
6506   lockdown   91.416592  639.566584   Topic7  -5.8011   0.0653
2542    country   89.015763  557.323245   Topic7  -5.8277   0.1763
12143       way   82.676466  427.099685   Topic7  -5.9016   0.3686
10377      stop   83.338572  793.427817   Topic7  -5.8936  -0.2428

[573 rows x 6 columns], token_table=       Topic      Freq                 Term
term                                       
7          3  0.264399               01_the
7          4  0.734441               01_the
86         3  0.122400                 1911
86         4  0.866218                 1911
87         3  0.984539              1911the
...      ...       ...                  ...
12501      4  0.173036                youre
12501      5  0.034040                youre
12501      6  0.036876                youre
12501      7  0.068080                youre
12510      6  0.985060  youwewillallbethere

[1577 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 1, 3, 2, 5, 4])

In [ ]:
!pip install pandas --upgrade

In [ ]:
pyLDAvis.save_html(panel,'/content/gdrive/My Drive/PhD/Program_Doc/Assignment/Project_Dataset/Topics_neg7'+ str(NUM_TOPICS) +'.html')

In [ ]:
#NMF

#@title Set topic modeling algorithm arguments

no_topics = 7 #@param {type:"integer"}

no_top_words = 5 #@param {type:"integer"}

no_top_documents = 4 #@param {type:"integer"}

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [ ]:
#@title Run NMF

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(str(doc_index) + ". " + documents[doc_index])

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data['Tokens'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

print("NMF Topics")
print(display_topics(nmf_H, nmf_W, tfidf_feature_names, data['Tokens'], no_top_words, no_top_documents))
#print("--------------")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://nump

NMF Topics
Topic 0:
people (3.96) vaccinated (2.17) need (1.28) stop (1.1) right (0.94)
11106. vaccinated people business freedom demand may turn away unvaccinated dangerous people cant force get vaccinated must still keep u safevaccinepassports
6841. work almost vulnerable people vaccinated need novaccinepassports
13136. injected people hence vaccinated people contagious around nonvaccinated people
17470. need change narrative letting people take mask next well see vaccinated people get sick due new strain blame unvaccinated people thats shrieking start
Topic 1:
vaccinemandates (1.56) medicaltyranny (1.53) angry (1.53) going (1.35) oligarchy (1.3)
9759. angry yet going vaccinemandates medicaltyranny plandemic republican democrat nothing protect god given constitutional right fightback oligarchy
9818. angry yet going vaccinemandates medicaltyranny plandemic republican democrat nothing protect god given constitutional right fightback oligarchy
9826. angry yet going vaccinemandates medic

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


In [ ]:
#Combine topics to original tweets
data["Topic N°"] = final_topics.argmax(axis=1)
data.head()

,username,author_location,text,created_at,Sentiment,compound,label,is_neg,is_neutral,is_pos,Text_punc,Text_stop,Text_tokenized,Text_lemmatized,Tokens,Text_lemmatized1,Topic N°
0,shalomegrace1,NaN,here's the war criminal pushing #vaccinepasspo...,2021-02-14 23:19:49+00:00,negative,-0.8399,-1.0,1.0,0.0,0.0,heres the war criminal pushing vaccinepassport...,heres war criminal pushing vaccinepassports ev...,"['heres', 'war', 'criminal', 'pushing', 'vacci...","['here', 'war', 'criminal', 'pushing', 'vaccin...",war criminal pushing everyday life going hard ...,here war criminal pushing vaccinepassports eve...,2
1,boomsbusts,@j0nathandavis for mkts/fin,"the sinister, huxlean background to #digitalpa...",2021-02-14 23:19:33+00:00,negative,-0.5994,-1.0,1.0,0.0,0.0,the sinister huxlean background to digitalpass...,sinister huxlean background digitalpassportsva...,"['sinister', 'huxlean', 'background', 'digital...","['sinister', 'huxlean', 'background', 'digital...",sinister huxlean background digitalpassportsva...,sinister huxlean background digitalpassportsva...,5
2,Artec7793,"California, USA",the agenda is obvious for those with eyes to s...,2021-02-14 23:12:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...,0
3,paragonlink,Éire,love the idea of losing our basic human rights...,2021-02-14 23:10:19+00:00,negative,-0.4871,-1.0,1.0,0.0,0.0,love the idea of losing our basic human rights...,love idea losing basic human rights twat go ha...,"['love', 'idea', 'losing', 'basic', 'human', '...","['love', 'idea', 'losing', 'basic', 'human', '...",love idea losing basic human right twat go hac...,love idea losing basic human right twat go hac...,0
4,midgard_misfit,Planet Designation: Earth-666,the agenda is obvious for those with eyes to s...,2021-02-14 23:10:18+00:00,negative,-0.5719,-1.0,1.0,0.0,0.0,the agenda is obvious for those with eyes to s...,agenda obvious eyes see check date hate right ...,"['agenda', 'obvious', 'eyes', 'see', 'check', ...","['agenda', 'obvious', 'eye', 'see', 'check', '...",agenda obvious eye see check date hate right,agenda obvious eye see check date hate right v...,0
